# Slack Channel Explorer

First, get all channels, then fetch their history.

In [ ]:
#whether to save jsons
jsons = False

#delete previous jsons
import os
import glob

def delete_jsons():
    json_files = glob.glob("jsons/*.json")
    if json_files:
        for file in json_files:
            os.remove(file)
        print(f"✓ Deleted {len(json_files)} JSON file(s)")
    else:
        print("✓ No JSON files to delete")

# Delete jsons when jsons = False
if not jsons:
    delete_jsons()

In [ ]:
import os
from dotenv import load_dotenv
import requests
import json
from datetime import datetime

load_dotenv()

USER_TOKEN = os.getenv("USER_TOKEN")

headers = {"Authorization": f"Bearer {USER_TOKEN}"}
#url = "https://slack.com/api/conversations.list"

url = "https://slack.com/api/conversations.history?channel=C1234567890"
response = requests.get(url, headers=headers)
data = response.json()

# # Create jsons folder if it doesn't exist
# os.makedirs("jsons", exist_ok=True)

# # Save with timestamp
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# filename = f"jsons/conversations_{timestamp}.json"

# with open(filename, "w", encoding="utf-8") as f:
#     json.dump(data, f, indent=2)

# print(f"Saved to: {filename}")
# print(json.dumps(data, indent=2))

In [ ]:
# Step 1: Get all channels
url = "https://slack.com/api/conversations.list"
response = requests.get(url, headers=headers)
channels_data = response.json()

# Save channels list only if jsons = True
if jsons:
    os.makedirs("jsons", exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"jsons/channels_list_{timestamp}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(channels_data, f, indent=2)
    print(f"Saved to: {filename}\n")

# Create dictionary of channel name to ID
channel_name_to_id = {}

if channels_data.get("ok"):
    channels = channels_data.get("channels", [])
    print(f"Found {len(channels)} channels:\n")
    for channel in channels:
        name = channel.get('name', 'N/A')
        channel_id = channel.get('id', 'N/A')
        channel_name_to_id[name] = channel_id
        print(f"  • {name:30} | ID: {channel_id}")
else:
    print("Error:", channels_data.get("error", "Unknown error"))
    print(json.dumps(channels_data, indent=2))

print(f"\n✓ Channel dictionary created with {len(channel_name_to_id)} channels")

In [ ]:
def get_channel_messages(channel_name, limit=100):
    """
    Extract messages from a Slack channel by channel name.
    
    Args:
        channel_name: The name of the channel (e.g., 'general', 'random')
        limit: Maximum number of messages to retrieve (default: 100)
    
    Returns:
        dict: The full API response containing messages and metadata
    """
    # Get channel ID from name
    if channel_name not in channel_name_to_id:
        print(f"Error: Channel '{channel_name}' not found!")
        print(f"Available channels: {', '.join(sorted(channel_name_to_id.keys()))}")
        return None
    
    channel_id = channel_name_to_id[channel_name]
    
    # Fetch conversation history
    url = f"https://slack.com/api/conversations.history"
    params = {
        "channel": channel_id,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    # Save to file only if jsons = True
    if jsons:
        os.makedirs("jsons", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"jsons/{channel_name}_history_{timestamp}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f"✓ Saved to: {filename}")
    
    # Display summary
    if data.get("ok"):
        messages = data.get("messages", [])
        print(f"✓ Retrieved {len(messages)} messages from #{channel_name}")
    else:
        print(f"✗ Error: {data.get('error', 'Unknown error')}")
    
    return data

print("✓ Function 'get_channel_messages' created and ready to use!")

In [ ]:
# Example: Get messages from a specific channel
# Replace 'general' with any channel name from the list above
result = get_channel_messages('all-astralsearch', limit=50)

# Display first few messages if successful

def head(result, n=3): #like pd.head()
    if result and result.get("ok"):
        messages = result.get("messages", [])
        print(f"\nFirst {n} messages preview:")
        for i, msg in enumerate(messages[:n], 1):
            text = msg.get("text", "")[:100]  # First 100 chars
            user = msg.get("user", "Unknown")
            print(f"\n{i}. User: {user}")
            print(f"   Text: {text}...")

head(result, n=5)

In [ ]:
def get_message_permalink(channel_name, message_ts, silent=True):
    """
    Get a permanent link to a specific Slack message.
    
    Args:
        channel_name: The name of the channel
        message_ts: The message timestamp (ts field)
        silent: If True, don't print errors (default: True)
    
    Returns:
        str: The permalink URL to the message, or None if error
    """
    if channel_name not in channel_name_to_id:
        return None
    
    channel_id = channel_name_to_id[channel_name]
    url = "https://slack.com/api/chat.getPermalink"
    params = {
        "channel": channel_id,
        "message_ts": message_ts
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data.get("ok"):
        return data.get("permalink")
    else:
        if not silent:
            print(f"Error getting permalink: {data.get('error')}")
        return None

def head_with_links(result, channel_name, n=3):
    """Display first n messages with their IDs and permalinks."""
    if result and result.get("ok"):
        messages = result.get("messages", [])
        print(f"\nFirst {n} messages with details:\n")
        
        for i, msg in enumerate(messages[:n], 1):
            text = msg.get("text", "")[:100]
            user = msg.get("user", "Unknown")
            msg_ts = msg.get("ts", "N/A")
            
            # Get permalink
            permalink = get_message_permalink(channel_name, msg_ts, silent=True)
            
            print(f"{i}. Message ID (ts): {msg_ts}")
            print(f"   User: {user}")
            print(f"   Text: {text}...")
            if permalink:
                print(f"   Link: {permalink}")
            print()  # blank line between messages

print("✓ Functions created: 'get_message_permalink' and 'head_with_links'")

In [ ]:
head_with_links(result, 'all-astralsearch', n=3)